<div align="center">

# Lump Sum vs Dollar Cost Averaging (DCA)
#### Should You Invest All at Once or Spread It Out?

<br>

</div>

You've just received a windfall — a bonus, an inheritance, or savings you've finally decided to invest. Now comes the classic question: **should you invest it all immediately, or spread it out over time?**

The first approach is **Lump Sum** — put everything in on day one and let it ride. The second is **Dollar Cost Averaging** — invest fixed amounts at regular intervals, like $1,000 per month for a year.

This notebook lets you explore both strategies using 53 years of real market data. You'll see which one wins more often, by how much, and why the answer isn't as simple as "always do X."

*Note: This analysis uses global equity data (MSCI World Index). Results may differ for bonds or other asset classes.*

In [ ]:
%pip install -q ipywidgets==8.1.8 ipecharts==1.4.0 pandas numpy

In [ ]:
import datetime
import numpy as np
import pandas as pd
from IPython.display import display, clear_output
from ipywidgets import (
    HTML,
    HBox,
    VBox,
    Layout,
    Button,
    Output,
    FloatText,
    DatePicker,
    IntText,
    Dropdown,
    IntProgress,
)
from ipecharts import EChartsRawWidget
from investment_utils import (
    BacktestParams,
    InvestmentResult,
    calculate_lump_sum,
    calculate_dca,
    run_multiple_backtests,
)
from market_data import load_msci_world

In [ ]:
COLORS = {
    "lump_sum": "#73c0de",
    "dca": "#ee6666",
    "success": "#3fa266",
    "danger": "#e34671",
}

# Fix button text alignment and add responsive card styles
display(
    HTML("""
<style>
.widget-button {
    display: flex !important;
    align-items: center !important;
    justify-content: center !important;
}
.widget-button i {
    margin-right: 6px;
}
/* Fix dropdown label cropping */
.widget-dropdown select {
    height: auto !important;
}
/* Stat cards (backtest results) */
.stat-cards-container {
    display: flex;
    flex-direction: row;
    gap: 20px;
    margin: 20px 0;
    justify-content: center;
    flex-wrap: wrap;
}
.stat-card {
    background: var(--jp-layout-color1, #f5f5f5);
    padding: 20px;
    border-radius: 8px;
    text-align: center;
    flex: 1 1 150px;
    min-width: 140px;
    max-width: 220px;
    box-sizing: border-box;
    border: 1px solid var(--jp-border-color2, rgba(0, 0, 0, 0.12));
    display: flex;
    flex-direction: column;
    justify-content: center;
}
/* Strategy cards (comparison results) */
.strategy-cards-container {
    display: flex;
    flex-direction: row;
    gap: 20px;
    margin: 20px 0;
    justify-content: center;
    flex-wrap: wrap;
}
.strategy-card {
    background: var(--jp-layout-color1, #f5f5f5);
    padding: 20px;
    border-radius: 8px;
    flex: 1 1 250px;
    min-width: 250px;
    max-width: 400px;
    box-sizing: border-box;
    border: 1px solid var(--jp-border-color2, rgba(0, 0, 0, 0.12));
}
@media (max-width: 600px) {
    .stat-cards-container {
        flex-direction: column;
        align-items: center;
    }
    .stat-card {
        width: 100%;
        max-width: 100%;
    }
    .strategy-cards-container {
        flex-direction: column;
        align-items: center;
    }
    .strategy-card {
        width: 100%;
        max-width: 100%;
    }
}
</style>
""")
)


def stat_card_html(
    title: str, value: str, subtitle: str = "", color: str = "#73c0de"
) -> str:
    """Return HTML string for a stat card (used in container)."""
    return f"""
        <div class="stat-card">
            <div style="color: var(--jp-ui-font-color2, #666); font-size: 14px;">{title}</div>
            <div style="font-size: 32px; color: {color}; margin: 10px 0; font-weight: bold;">{value}</div>
            <div style="color: var(--jp-ui-font-color3, #888); font-size: 12px;">{subtitle}</div>
        </div>
    """


def stat_cards_row(cards_html: list[str]) -> HTML:
    """Create a responsive row of stat cards."""
    return HTML(f"""
        <div class="stat-cards-container">
            {"".join(cards_html)}
        </div>
    """)


def metric_row(label: str, value: str, color: str = None) -> str:
    """Generate HTML for a metric row in a table."""
    style = f"color: {color};" if color else ""
    return (
        f'<tr><td>{label}</td><td style="text-align: right; {style}">{value}</td></tr>'
    )


def strategy_card_html(name: str, result: InvestmentResult, color: str) -> str:
    """Return HTML string for a strategy result card."""
    ret_color = COLORS["success"] if result.total_return >= 0 else COLORS["danger"]
    return f"""
        <div class="strategy-card" style="border-left: 4px solid {color};">
            <h3 style="color: {color}; margin: 0 0 15px 0;">{name}</h3>
            <table style="width: 100%; color: var(--jp-ui-font-color1, #333);">
                {metric_row("Invested", f"${result.total_invested:,.0f}")}
                {metric_row("Final Value", f"${result.final_value:,.0f}")}
                {metric_row("Total Return", f"{result.total_return_pct:+.2f}%", ret_color)}
                {metric_row("Annualized", f"{result.annualized_return:.2f}%")}
                {metric_row("Max Drawdown", f"-{result.max_drawdown:.2f}%", COLORS["danger"])}
                {metric_row("Volatility", f"{result.volatility:.2f}%")}
                {metric_row("Sharpe Ratio", f"{result.sharpe_ratio:.2f}")}
            </table>
        </div>
    """


def strategy_cards_row(cards_html: list[str]) -> HTML:
    """Create a responsive row of strategy cards."""
    return HTML(f"""
        <div class="strategy-cards-container">
            {"".join(cards_html)}
        </div>
    """)

## 1. The Data: MSCI World Index (1972–2025)

We'll use the **MSCI World Index**, which tracks large and mid-cap stocks across 23 developed countries. It's one of the most widely used benchmarks for global equity performance.

This gives us **53 years** of market history — including the 1970s stagflation, 1987 Black Monday, the dot-com bubble, the 2008 financial crisis, COVID-19, and everything in between.

In [ ]:
def create_price_chart(prices, title):
    """Create a price chart using ipecharts."""
    if len(prices) > 2000:
        step = len(prices) // 2000
        prices_sampled = prices.iloc[::step]
    else:
        prices_sampled = prices

    dates = [d.strftime("%Y-%m-%d") for d in prices_sampled.index]
    values = [float(v) for v in prices_sampled.values]

    option = {
        "backgroundColor": "transparent",
        "title": {
            "text": title,
            "left": "center",
            "textStyle": {"width": "90%", "overflow": "truncate"},
        },
        "toolbox": {"feature": {"restore": {"title": "Reset Zoom"}}},
        "tooltip": {"trigger": "axis", "axisPointer": {"type": "cross"}},
        "xAxis": {"type": "category", "data": dates},
        "yAxis": {"type": "value", "name": "Price"},
        "series": [
            {
                "name": title,
                "type": "line",
                "data": values,
                "smooth": True,
                "symbol": "none",
                "areaStyle": {"opacity": 0.15},
            }
        ],
        "grid": {"left": "15%", "right": "5%", "bottom": "20%", "top": "15%"},
        "dataZoom": [
            {"type": "slider", "xAxisIndex": 0, "start": 0, "end": 100, "bottom": "5%"}
        ],
    }

    chart = EChartsRawWidget(option=option, style={"height": "400px", "width": "100%"})
    display(chart)


PRICES = load_msci_world()
create_price_chart(PRICES, "MSCI World Index (1972-2025)")

## 2. Single Period Comparison

Pick a start date and investment amount to see how each strategy would have performed. This simulates what would happen if you had invested at that specific time.

**How to read the results:**
- **Lump Sum** invests the full amount on day one
- **DCA** spreads purchases over the specified duration (e.g., 12 monthly installments)
- Both strategies then hold until the end of the investment horizon

In [ ]:
style = {"description_width": "180px"}
layout = Layout(width="100%")

investment_amount = FloatText(
    value=10000, description="Total Amount ($):", step=1000, style=style, layout=layout
)
start_date_input = DatePicker(
    value=datetime.date(1990, 1, 1),
    description="Start Date:",
    style=style,
    layout=layout,
)
investment_horizon = IntText(
    value=20, description="Investment Horizon (years):", style=style, layout=layout
)
dca_frequency_selector = Dropdown(
    options=[("Weekly", "weekly"), ("Monthly", "monthly"), ("Quarterly", "quarterly")],
    value="monthly",
    description="DCA Frequency:",
    style=style,
    layout=layout,
)
dca_duration_input = IntText(
    value=12, description="DCA Duration (months):", style=style, layout=layout
)

run_button = Button(
    description="Compare Strategies",
    button_style="primary",
    icon="chart-line",
    layout=Layout(width="100%", margin="10px 0"),
)
output_area = Output()


def create_portfolio_chart(ls_result, dca_result, freq):
    """Create portfolio value comparison chart."""
    dates = [d.strftime("%Y-%m-%d") for d in ls_result.timeline.index]
    option = {
        "backgroundColor": "transparent",
        "title": {
            "text": "Portfolio Value Comparison",
            "left": "center",
            "textStyle": {"width": "90%", "overflow": "truncate"},
        },
        "toolbox": {"feature": {"restore": {"title": "Reset Zoom"}}},
        "tooltip": {"trigger": "axis", "axisPointer": {"type": "cross"}},
        "legend": {
            "data": ["Lump Sum", f"DCA ({freq.capitalize()})", "DCA Amount Invested"],
            "bottom": 0,
        },
        "xAxis": {"type": "category", "data": dates, "boundaryGap": False},
        "yAxis": {"type": "value", "name": "Portfolio Value ($)"},
        "series": [
            {
                "name": "Lump Sum",
                "type": "line",
                "data": [float(v) for v in ls_result.timeline["value"]],
                "smooth": True,
                "symbol": "none",
            },
            {
                "name": f"DCA ({freq.capitalize()})",
                "type": "line",
                "data": [float(v) for v in dca_result.timeline["value"]],
                "smooth": True,
                "symbol": "none",
            },
            {
                "name": "DCA Amount Invested",
                "type": "line",
                "data": [float(v) for v in dca_result.timeline["invested"]],
                "smooth": True,
                "symbol": "none",
                "lineStyle": {"type": "dotted", "opacity": 0.6},
            },
        ],
        "grid": {"left": "15%", "right": "5%", "bottom": "20%", "top": "15%"},
        "dataZoom": [
            {"type": "slider", "xAxisIndex": 0, "start": 0, "end": 100, "bottom": "10%"}
        ],
    }
    return EChartsRawWidget(option=option, style={"height": "550px", "width": "100%"})


def run_single_comparison(btn):
    with output_area:
        clear_output(wait=True)

        try:
            total = investment_amount.value
            start = start_date_input.value.strftime("%Y-%m-%d")
            horizon = investment_horizon.value
            freq = dca_frequency_selector.value
            dca_months = dca_duration_input.value

            start_dt = pd.to_datetime(start)
            end_dt = start_dt + pd.DateOffset(years=horizon)
            end = end_dt.strftime("%Y-%m-%d")

            params_ls = BacktestParams(
                total_amount=total, start_date=start, end_date=end
            )
            params_dca = BacktestParams(
                total_amount=total,
                start_date=start,
                end_date=end,
                dca_frequency=freq,
                dca_periods=dca_months if freq == "monthly" else None,
            )

            ls_result = calculate_lump_sum(PRICES, params_ls)
            dca_result = calculate_dca(PRICES, params_dca)

            winner = (
                "Lump Sum" if ls_result.final_value > dca_result.final_value else "DCA"
            )
            diff = abs(ls_result.final_value - dca_result.final_value)
            diff_pct = abs(ls_result.total_return_pct - dca_result.total_return_pct)
            winner_color = COLORS["lump_sum"] if winner == "Lump Sum" else COLORS["dca"]

            winner_banner = HTML(f"""
                <div style="text-align: center; padding: 20px; background: var(--jp-layout-color1, #f5f5f5);
                            border-radius: 12px; border: 1px solid var(--jp-border-color2, rgba(0, 0, 0, 0.12)); margin-bottom: 20px;">
                    <span style="font-size: 18px; color: var(--jp-ui-font-color2, #666);">Winner:</span>
                    <h1 style="color: {winner_color}; margin: 5px 0;">{winner}</h1>
                    <span style="font-size: 14px; color: var(--jp-ui-font-color2, #666);">
                        by ${diff:,.0f} ({diff_pct:.2f}% better return)
                    </span>
                </div>
            """)

            cards_row = strategy_cards_row(
                [
                    strategy_card_html("Lump Sum", ls_result, COLORS["lump_sum"]),
                    strategy_card_html(
                        f"DCA ({freq.capitalize()})", dca_result, COLORS["dca"]
                    ),
                ]
            )

            display(
                HTML(
                    f'<h2 style="color: var(--jp-ui-font-color1, #333);">Results: MSCI World ({start} to {end})</h2>'
                )
            )
            display(winner_banner)
            display(cards_row)
            display(
                HTML(
                    '<h3 style="color: var(--jp-ui-font-color1, #333); margin-top: 30px;">Portfolio Value Over Time</h3>'
                )
            )
            display(create_portfolio_chart(ls_result, dca_result, freq))

        except Exception as e:
            display(
                HTML(f"""
                <div style="color: {COLORS["danger"]}; padding: 20px; background: rgba(227, 70, 113, 0.1); border-radius: 8px;">
                    <strong>Error:</strong> {str(e)}
                </div>
            """)
            )
            import traceback

            traceback.print_exc()


run_button.on_click(run_single_comparison)

inputs_panel = VBox(
    [
        HTML('<h3 style="color: #2E86AB; margin-top: 0;">Investment Parameters</h3>'),
        investment_amount,
        start_date_input,
        investment_horizon,
        HTML('<h4 style="color: #A23B72; margin-top: 15px;">DCA Settings</h4>'),
        dca_frequency_selector,
        dca_duration_input,
        run_button,
    ],
    layout=Layout(
        padding="15px",
        max_width="450px",
        width="100%",
        border="1px solid var(--jp-border-color2, #ddd)",
        border_radius="8px",
    ),
)

placeholder_html = HTML("""
    <div style="min-height: 400px; display: flex; align-items: center; justify-content: center;
                color: var(--jp-ui-font-color3, #888); font-style: italic;">
        Click "Compare Strategies" to see results
    </div>
""")
with output_area:
    display(placeholder_html)

results_panel = VBox(
    [output_area],
    layout=Layout(
        width="100%",
        min_height="450px",
        border="1px solid var(--jp-border-color2, #ddd)",
        border_radius="8px",
        padding="15px",
    ),
)

VBox(
    [
        HBox([inputs_panel], layout=Layout(justify_content="center", width="100%")),
        results_panel,
    ],
    layout=Layout(gap="20px"),
)

## 3. Historical Backtest

One period doesn't tell the whole story. What if you picked the wrong time to start?

This section answers: **"Across all possible starting points in history, how often does each strategy win?"**

We run hundreds of backtests — one for each possible start date — to see how consistently each strategy performs. This is called a **rolling window analysis** and is the same methodology used in academic research.

In [ ]:
bt_style = {"description_width": "180px"}
bt_layout = Layout(width="100%")

bt_amount = FloatText(
    value=10000,
    description="Investment Amount ($):",
    step=1000,
    style=bt_style,
    layout=bt_layout,
)
bt_horizon = IntText(
    value=20, description="Investment Horizon (yrs):", style=bt_style, layout=bt_layout
)
bt_dca_months = IntText(
    value=12, description="DCA Period (months):", style=bt_style, layout=bt_layout
)

bt_button = Button(
    description="Run Backtests",
    button_style="success",
    icon="play",
    layout=Layout(width="100%", margin="10px 0"),
)
bt_output = Output()


def create_backtest_bar_chart(results, horizon):
    """Create backtest bar chart."""
    dates = [d.strftime("%Y-%m") for d in results["start_date"]]
    values = [float(v) for v in results["difference_pct"]]
    bar_data = [
        {"value": v, "itemStyle": {"color": COLORS["lump_sum"] if w else COLORS["dca"]}}
        for v, w in zip(values, results["ls_wins"])
    ]
    option = {
        "backgroundColor": "transparent",
        "title": {
            "text": "LS vs DCA: Return Difference",
            "subtext": f"Each bar = one {horizon}-year period",
            "left": "center",
            "textStyle": {"width": "90%", "overflow": "truncate"},
        },
        "toolbox": {"feature": {"restore": {"title": "Reset Zoom"}}},
        "tooltip": {"trigger": "axis", "axisPointer": {"type": "shadow"}},
        "xAxis": {"type": "category", "data": dates, "axisLabel": {"rotate": 45}},
        "yAxis": {"type": "value", "name": "LS - DCA (%)"},
        "series": [{"name": "LS Advantage", "type": "bar", "data": bar_data}],
        "grid": {"left": "15%", "right": "5%", "bottom": "25%", "top": "20%"},
        "dataZoom": [
            {"type": "slider", "xAxisIndex": 0, "start": 0, "end": 100, "bottom": "5%"}
        ],
    }
    return EChartsRawWidget(option=option, style={"height": "450px", "width": "100%"})


def create_histogram_chart(results, horizon):
    """Create histogram chart."""
    ls_returns = results["ls_return_pct"].values
    dca_returns = results["dca_return_pct"].values
    all_returns = np.concatenate([ls_returns, dca_returns])
    min_val = np.floor(all_returns.min() / 10) * 10
    max_val = np.ceil(all_returns.max() / 10) * 10
    bins = np.linspace(min_val, max_val, 21)
    ls_counts, _ = np.histogram(ls_returns, bins=bins)
    dca_counts, _ = np.histogram(dca_returns, bins=bins)
    bin_labels = [f"{int(bins[i])}-{int(bins[i + 1])}%" for i in range(len(bins) - 1)]

    option = {
        "backgroundColor": "transparent",
        "title": {
            "text": f"{horizon}-Year Returns Distribution",
            "left": "center",
            "textStyle": {"width": "90%", "overflow": "truncate"},
        },
        "toolbox": {"feature": {"restore": {"title": "Reset Zoom"}}},
        "tooltip": {"trigger": "axis", "axisPointer": {"type": "shadow"}},
        "legend": {"data": ["Lump Sum", "DCA"], "bottom": 0},
        "xAxis": {
            "type": "category",
            "data": bin_labels,
            "name": "Return (%)",
            "axisLabel": {"rotate": 45},
        },
        "yAxis": {"type": "value", "name": "Periods"},
        "series": [
            {"name": "Lump Sum", "type": "bar", "data": [int(c) for c in ls_counts]},
            {"name": "DCA", "type": "bar", "data": [int(c) for c in dca_counts]},
        ],
        "grid": {"left": "15%", "right": "5%", "bottom": "25%", "top": "15%"},
    }
    return EChartsRawWidget(option=option, style={"height": "400px", "width": "100%"})


def run_backtests(btn):
    bt_button.disabled = True

    with bt_output:
        clear_output(wait=True)

        try:
            total = bt_amount.value
            horizon = bt_horizon.value
            dca_months = bt_dca_months.value

            min_date = PRICES.index.min()
            max_date = PRICES.index.max()
            max_start = max_date - pd.DateOffset(years=horizon)

            if max_start <= min_date:
                print(f"Not enough data for {horizon}-year backtests.")
                bt_button.disabled = False
                return

            start_dates = pd.date_range(start=min_date, end=max_start, freq="MS")
            start_dates_str = [d.strftime("%Y-%m-%d") for d in start_dates]

            progress = IntProgress(
                value=0,
                min=0,
                max=len(start_dates_str),
                description="Running:",
                bar_style="info",
                style={"bar_color": "#3fa266", "description_width": "80px"},
                layout=Layout(width="100%"),
            )
            progress_label = HTML(
                f'<span style="color: var(--jp-ui-font-color2, #666);">0 / {len(start_dates_str)} backtests</span>'
            )
            progress_container = VBox(
                [progress, progress_label],
                layout=Layout(margin="10px 0"),
            )
            display(progress_container)

            def update_progress(current, total_count):
                progress.value = current
                progress_label.value = f'<span style="color: var(--jp-ui-font-color2, #666);">{current} / {total_count} backtests</span>'

            results = run_multiple_backtests(
                prices=PRICES,
                total_amount=total,
                start_dates=start_dates_str,
                investment_horizon_years=horizon,
                dca_frequency="monthly",
                dca_duration_months=dca_months,
                progress_callback=update_progress,
            )

            clear_output(wait=True)

            if len(results) == 0:
                print("No valid backtests could be run.")
                return

            results["start_date"] = pd.to_datetime(results["start_date"])
            results["end_date"] = results["start_date"] + pd.DateOffset(years=horizon)

            ls_wins = results["ls_wins"].sum()
            dca_wins = len(results) - ls_wins
            ls_win_pct = (ls_wins / len(results)) * 100
            avg_diff = results["difference_pct"].mean()

            first_start = results["start_date"].min().strftime("%Y")
            last_start = results["start_date"].max().strftime("%Y")
            last_end = results["end_date"].max().strftime("%Y")

            display(
                HTML(
                    '<h2 style="color: var(--jp-ui-font-color1, #333);">Backtest Results: MSCI World</h2>'
                )
            )
            display(
                HTML(f"""
                <p style="color: var(--jp-ui-font-color1, #333);"><b>{len(results)} rolling {horizon}-year periods tested</b></p>
                <p style="color: var(--jp-ui-font-color2, #666); font-size: 13px;">
                    Start dates: {first_start} to {last_start} | End dates: {int(first_start) + horizon} to {last_end}
                </p>
            """)
            )

            cards = stat_cards_row(
                [
                    stat_card_html(
                        "Lump Sum Wins",
                        f"{ls_win_pct:.1f}%",
                        f"{ls_wins} of {len(results)}",
                        COLORS["lump_sum"],
                    ),
                    stat_card_html(
                        "DCA Wins",
                        f"{100 - ls_win_pct:.1f}%",
                        f"{dca_wins} of {len(results)}",
                        COLORS["dca"],
                    ),
                    stat_card_html(
                        "Avg LS Advantage",
                        f"{avg_diff:+.1f}%",
                        "",
                        COLORS["success"] if avg_diff > 0 else COLORS["danger"],
                    ),
                ]
            )
            display(cards)

            display(create_backtest_bar_chart(results, horizon))
            display(create_histogram_chart(results, horizon))

        except Exception as e:
            print(f"Error: {e}")
            import traceback

            traceback.print_exc()
        finally:
            bt_button.disabled = False


bt_button.on_click(run_backtests)

bt_inputs_panel = VBox(
    [
        HTML('<h3 style="color: #2E86AB; margin-top: 0;">Backtest Parameters</h3>'),
        bt_amount,
        bt_horizon,
        bt_dca_months,
        HTML(
            '<p style="color: var(--jp-ui-font-color3, #888); font-size: 12px; margin: 10px 0;">Runs backtests starting from each month in the available data.</p>'
        ),
        bt_button,
    ],
    layout=Layout(
        padding="15px",
        max_width="450px",
        width="100%",
        border="1px solid var(--jp-border-color2, #ddd)",
        border_radius="8px",
    ),
)

bt_placeholder_html = HTML("""
    <div style="min-height: 400px; display: flex; align-items: center; justify-content: center;
                color: var(--jp-ui-font-color3, #888); font-style: italic;">
        Click "Run Backtests" to see results
    </div>
""")
with bt_output:
    display(bt_placeholder_html)

bt_results_panel = VBox(
    [bt_output],
    layout=Layout(
        width="100%",
        min_height="450px",
        border="1px solid var(--jp-border-color2, #ddd)",
        border_radius="8px",
        padding="15px",
    ),
)

VBox(
    [
        HBox([bt_inputs_panel], layout=Layout(justify_content="center", width="100%")),
        bt_results_panel,
    ],
    layout=Layout(gap="20px"),
)

## Conclusion

**Lump Sum wins about two-thirds of the time.** Investing immediately beats spreading it out in roughly 66% of historical periods. This matches findings from [Vanguard's 2012 study](https://static.twentyoverten.com/5980d16bbfb1c93238ad9c24/rJpQmY8o7/Dollar-Cost-Averaging-Just-Means-Taking-Risk-Later-Vanguard.pdf), which analyzed 85 years of U.S. data and multiple international markets.

Why? Markets trend upward over time. DCA leaves cash uninvested, missing potential gains. The longer your DCA period, the more returns you sacrifice.

But DCA isn't wrong. While Lump Sum delivers higher expected returns, DCA provides a smoother ride. In the Vanguard study, Lump Sum portfolios experienced negative 10-year returns 22% of the time, compared to just 18% for DCA. If sleeping well matters more than maximizing every dollar, that tradeoff can be worth it.

DCA makes sense if you can't stomach investing everything at once, you're investing from each paycheck anyway, or you want protection against the regret of bad timing.

If you have the money now and a long time horizon, the math favors investing it now. But if DCA is what gets you to actually invest instead of sitting in cash, it's the better choice *for you*. Both strategies beat not investing at all.

*Past performance does not guarantee future results. This is for educational purposes only, not financial advice.*

---
*Data: MSCI World Index (1972–2025) | Research: [Vanguard (2012)](https://static.twentyoverten.com/5980d16bbfb1c93238ad9c24/rJpQmY8o7/Dollar-Cost-Averaging-Just-Means-Taking-Risk-Later-Vanguard.pdf)*